# JOIN()

In [14]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import explode, col

spark = (
    SparkSession.builder
    .appName("example-join")
    .getOrCreate()
)

In [17]:
# Emp Table
empData = [(1,"Smith",10), (2,"Rose",20),(3,"Williams",10), (4,"Jones",30)]

empColumns = ["emp_id","name","emp_dept_id"]

empDF = spark.createDataFrame(empData,empColumns)
empDF.show()

+------+--------+-----------+
|emp_id|    name|emp_dept_id|
+------+--------+-----------+
|     1|   Smith|         10|
|     2|    Rose|         20|
|     3|Williams|         10|
|     4|   Jones|         30|
+------+--------+-----------+



In [18]:
# Dept Table
deptData = [("Finance",10), ("Marketing",20), ("Sales",30),("IT",40)]

deptColumns = ["dept_name","dept_id"]

deptDF=spark.createDataFrame(deptData,deptColumns)  
deptDF.show()

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|  Finance|     10|
|Marketing|     20|
|    Sales|     30|
|       IT|     40|
+---------+-------+



In [19]:
# Address Table
addData=[(1,"1523 Main St","SFO","CA"),
         (2,"3453 Orange St","SFO","NY"),
         (3,"34 Warner St","Jersey","NJ"),
         (4,"221 Cavalier St","Newark","DE"),
         (5,"789 Walnut St","Sandiago","CA")]

addColumns = ["emp_id", "addline1", "city", "state"]

addDF = spark.createDataFrame(addData,addColumns)
addDF.show()

+------+---------------+--------+-----+
|emp_id|       addline1|    city|state|
+------+---------------+--------+-----+
|     1|   1523 Main St|     SFO|   CA|
|     2| 3453 Orange St|     SFO|   NY|
|     3|   34 Warner St|  Jersey|   NJ|
|     4|221 Cavalier St|  Newark|   DE|
|     5|  789 Walnut St|Sandiago|   CA|
+------+---------------+--------+-----+



### PySpark Join Two DataFrames

In [20]:
# join(right, joinExprs, joinType)
# join(right)

empDF.join(addDF, empDF["emp_id"] == addDF["emp_id"]).show()

+------+--------+-----------+------+---------------+------+-----+
|emp_id|    name|emp_dept_id|emp_id|       addline1|  city|state|
+------+--------+-----------+------+---------------+------+-----+
|     1|   Smith|         10|     1|   1523 Main St|   SFO|   CA|
|     3|Williams|         10|     3|   34 Warner St|Jersey|   NJ|
|     2|    Rose|         20|     2| 3453 Orange St|   SFO|   NY|
|     4|   Jones|         30|     4|221 Cavalier St|Newark|   DE|
+------+--------+-----------+------+---------------+------+-----+



In [21]:
# Drop Duplicate Columns After Join
empDF.join(addDF,["emp_id"]).show()

+------+--------+-----------+---------------+------+-----+
|emp_id|    name|emp_dept_id|       addline1|  city|state|
+------+--------+-----------+---------------+------+-----+
|     1|   Smith|         10|   1523 Main St|   SFO|   CA|
|     3|Williams|         10|   34 Warner St|Jersey|   NJ|
|     2|    Rose|         20| 3453 Orange St|   SFO|   NY|
|     4|   Jones|         30|221 Cavalier St|Newark|   DE|
+------+--------+-----------+---------------+------+-----+



In [22]:
#Join Multiple DataFrames by chaining
empDF.join(addDF,["emp_id"]) \
     .join(deptDF,empDF["emp_dept_id"] == deptDF["dept_id"]) \
     .show()

+------+--------+-----------+---------------+------+-----+---------+-------+
|emp_id|    name|emp_dept_id|       addline1|  city|state|dept_name|dept_id|
+------+--------+-----------+---------------+------+-----+---------+-------+
|     1|   Smith|         10|   1523 Main St|   SFO|   CA|  Finance|     10|
|     3|Williams|         10|   34 Warner St|Jersey|   NJ|  Finance|     10|
|     4|   Jones|         30|221 Cavalier St|Newark|   DE|    Sales|     30|
|     2|    Rose|         20| 3453 Orange St|   SFO|   NY|Marketing|     20|
+------+--------+-----------+---------------+------+-----+---------+-------+



In [23]:
# Using Where for Join Condition
empDF.join(deptDF).where(empDF["emp_dept_id"] == deptDF["dept_id"]) \
    .join(addDF).where(empDF["emp_id"] == addDF["emp_id"]) \
    .show()

+------+--------+-----------+---------+-------+------+---------------+------+-----+
|emp_id|    name|emp_dept_id|dept_name|dept_id|emp_id|       addline1|  city|state|
+------+--------+-----------+---------+-------+------+---------------+------+-----+
|     1|   Smith|         10|  Finance|     10|     1|   1523 Main St|   SFO|   CA|
|     3|Williams|         10|  Finance|     10|     3|   34 Warner St|Jersey|   NJ|
|     2|    Rose|         20|Marketing|     20|     2| 3453 Orange St|   SFO|   NY|
|     4|   Jones|         30|    Sales|     30|     4|221 Cavalier St|Newark|   DE|
+------+--------+-----------+---------+-------+------+---------------+------+-----+



In [24]:
# Using Filter for Join Condition
empDF.join(deptDF).filter(empDF["emp_dept_id"] == deptDF["dept_id"]) \
    .join(addDF).filter(empDF["emp_id"] == addDF["emp_id"]) \
    .show()

+------+--------+-----------+---------+-------+------+---------------+------+-----+
|emp_id|    name|emp_dept_id|dept_name|dept_id|emp_id|       addline1|  city|state|
+------+--------+-----------+---------+-------+------+---------------+------+-----+
|     1|   Smith|         10|  Finance|     10|     1|   1523 Main St|   SFO|   CA|
|     3|Williams|         10|  Finance|     10|     3|   34 Warner St|Jersey|   NJ|
|     2|    Rose|         20|Marketing|     20|     2| 3453 Orange St|   SFO|   NY|
|     4|   Jones|         30|    Sales|     30|     4|221 Cavalier St|Newark|   DE|
+------+--------+-----------+---------+-------+------+---------------+------+-----+



In [25]:
# SQL
empDF.createOrReplaceTempView("emp")
deptDF.createOrReplaceTempView("dept")
addDF.createOrReplaceTempView("add")

spark.sql("""
             select * 
               from emp e, dept d, add a
              where e.emp_dept_id == d.dept_id 
                and e.emp_id == a.emp_id
         """) \
    .show()

+------+--------+-----------+---------+-------+------+---------------+------+-----+
|emp_id|    name|emp_dept_id|dept_name|dept_id|emp_id|       addline1|  city|state|
+------+--------+-----------+---------+-------+------+---------------+------+-----+
|     1|   Smith|         10|  Finance|     10|     1|   1523 Main St|   SFO|   CA|
|     3|Williams|         10|  Finance|     10|     3|   34 Warner St|Jersey|   NJ|
|     2|    Rose|         20|Marketing|     20|     2| 3453 Orange St|   SFO|   NY|
|     4|   Jones|         30|    Sales|     30|     4|221 Cavalier St|Newark|   DE|
+------+--------+-----------+---------+-------+------+---------------+------+-----+



In [26]:
# PySpark Join With Multiple Columns & Conditions
df1 = spark.createDataFrame([(1, "A"), (2, "B"), (3, "C")], ["A1", "A2"])

df2 = spark.createDataFrame([(1, "F"), (2, "B")], ["B1", "B2"])

df = df1.join(df2, (df1.A1 == df2.B1) & (df1.A2 == df2.B2))
df.show()

+---+---+---+---+
| A1| A2| B1| B2|
+---+---+---+---+
|  2|  B|  2|  B|
+---+---+---+---+



The code block displayed below contains an error. The code block is intended to perform an outer join of DataFrames transactionsDf and itemsDf on columns productId and itemId, respectively. Find the error.

Code block:

transactionsDf.join(itemsDf, [itemsDf.itemId, transactionsDf.productId], “outer”)

- The "outer" argument should be eliminated, since "outer" is the default join type.
- The join type needs to be appended to the join() operator, like join().outer() instead of listing it as the last argument inside the join() call.
- The term [itemsDf.itemId, transactionsDf.productId] should be replaced by itemsDf.itemId == transactionsDf.productId.
- The term [itemsDf.itemId, transactionsDf.productId] should be replaced by itemsDf.col("itemId") == transactionsDf.col("productId").
- The "outer" argument should be eliminated from the call and join should be replaced by joinOuter.

In [27]:
# Error [df1.A1, df2.B1]
df1.join(df2, [df1.A1, df2.B1], "outer").show()

AnalysisException: cannot resolve '(`A1` AND `B1`)' due to data type mismatch: '(`A1` AND `B1`)' requires boolean type, not bigint;;
'Join FullOuter, (A1#831L AND B1#835L)
:- LogicalRDD [A1#831L, A2#832], false
+- LogicalRDD [B1#835L, B2#836], false


In [28]:
# Correct 
cond = [df1.A1 == df2.B1]
df1.join(df2, cond, 'outer').show()


+---+---+----+----+
| A1| A2|  B1|  B2|
+---+---+----+----+
|  1|  A|   1|   F|
|  3|  C|null|null|
|  2|  B|   2|   B|
+---+---+----+----+



In [3]:
data = [(1, 'Thick Coat for Walking in the Snow', ['blue', 'winter', 'cozy'], 'Sports Company Inc.'),
        (2, 'Elegant Outdoors Summer Dress', ['red', 'summer'], 'YetiX'),
        (3, 'Outdoors Backpack', ['green', 'summer'], 'Sports Company Inc.')]

columns = ["itemId", "itemName", "attributes", "supplier"]

itemsDf = spark.createDataFrame(data=data, schema=columns)

itemsDf.printSchema()
itemsDf.show()

root
 |-- itemId: long (nullable = true)
 |-- itemName: string (nullable = true)
 |-- attributes: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- supplier: string (nullable = true)

+------+--------------------+--------------------+-------------------+
|itemId|            itemName|          attributes|           supplier|
+------+--------------------+--------------------+-------------------+
|     1|Thick Coat for Wa...|[blue, winter, cozy]|Sports Company Inc.|
|     2|Elegant Outdoors ...|       [red, summer]|              YetiX|
|     3|   Outdoors Backpack|     [green, summer]|Sports Company Inc.|
+------+--------------------+--------------------+-------------------+



In [4]:
data = [(1, 3, 4, 25, 1, None, 1587915332),
         (2, 6, 7, 2, 2, None, 1586815312),
         (3, 3, None, 25, 3, None, 1585824821),
         (4, None, None, 3, 2, None, 1583244275),
         (5, None, None, None, 2, None, 1575285427),
         (6, 3, 2, 25, 2, None, 1572733275)]

schema = StructType([StructField('transactionId', IntegerType(), True),
                     StructField('predError', IntegerType(), True),
                     StructField('value', IntegerType(), True),
                     StructField('storeId', IntegerType(), True),
                     StructField('productId', IntegerType(), True),
                     StructField('f', IntegerType(), True),
                     StructField('transactionDate', LongType(), True)])

transactionsDf = spark.createDataFrame(data=data, schema=schema)

In [17]:
itemsDf.join(transactionsDf, itemsDf.itemId == transactionsDf.transactionId, "inner").show()

+------+--------------------+--------------------+-------------------+-------------+---------+-----+-------+---------+----+---------------+
|itemId|            itemName|          attributes|           supplier|transactionId|predError|value|storeId|productId|   f|transactionDate|
+------+--------------------+--------------------+-------------------+-------------+---------+-----+-------+---------+----+---------------+
|     1|Thick Coat for Wa...|[blue, winter, cozy]|Sports Company Inc.|            1|        3|    4|     25|        1|null|     1587915332|
|     3|   Outdoors Backpack|     [green, summer]|Sports Company Inc.|            3|        3| null|     25|        3|null|     1585824821|
|     2|Elegant Outdoors ...|       [red, summer]|              YetiX|            2|        6|    7|      2|        2|null|     1586815312|
+------+--------------------+--------------------+-------------------+-------------+---------+-----+-------+---------+----+---------------+



In [5]:
# AssertionError: how should be basestring
itemsDf.join(transactionsDf, "inner", itemsDf.itemId == transactionsDf.transactionId)

# NameError: name 'itemId' is not defined
itemsDf.join(transactionsDf, itemId == transactionId)

# AnalysisException: USING column `itemsDf.itemId == transactionsDf.transactionId` 
# cannot be resolved on the left side of the join. 
# The left-side columns: [itemId, itemName, attributes, supplier];
itemsDf.join(transactionsDf, "itemsDf.itemId == transactionsDf.transactionId", "inner")


# Py4JError: An error occurred while calling z:org.apache.spark.sql.functions.col. Trace:
# py4j.Py4JException: Method col([class org.apache.spark.sql.Column]) does not exist
itemsDf.join(transactionsDf, col(itemsDf.itemId) == col(transactionsDf.transactionId))

AssertionError: how should be basestring

+------+----------------------------------+--------------------+-------------------+-------------+---------+-----+-------+---------+----+---------------+
|itemId|itemName                          |attributes          |supplier           |transactionId|predError|value|storeId|productId|f   |transactionDate|
+------+----------------------------------+--------------------+-------------------+-------------+---------+-----+-------+---------+----+---------------+
|1     |Thick Coat for Walking in the Snow|[blue, winter, cozy]|Sports Company Inc.|1            |3        |4    |25     |1        |null|1587915332     |
|3     |Outdoors Backpack                 |[green, summer]     |Sports Company Inc.|3            |3        |null |25     |3        |null|1585824821     |
|2     |Elegant Outdoors Summer Dress     |[red, summer]       |YetiX              |2            |6        |7    |2      |2        |null|1586815312     |
+------+----------------------------------+--------------------+------------